## EDA Hate Crimes Main Database

In [3]:
# Import necessary libraries
import pandas as pd

In [4]:
file_path = '/Users/sayo/personal_projects/Usafe_bot/data/hate_crimes.csv'  
df = pd.read_csv(file_path)

df.head()

,Date,Country,Bias motivations,Type of incident,Source,Description
0,2016-11,Belgium,Anti-Semitic hate crime,Violent attacks against people,Antisemitisme.be,A man visibly identifiable as Jewish was pushe...
1,2016-11,Belgium,Anti-Semitic hate crime,Threats,Antisemitisme.be,A man visibly identifiable as Jewish was chase...
2,2016-03,Belgium,Anti-Semitic hate crime,Violent attacks against people,"Anti-Defamation League (ADL), Antisemitisme.be",Stones were thrown at people visibly identifia...
3,2016-03,Belgium,Anti-Semitic hate crime,Violent attacks against people,Antisemitisme.be,Stones were thrown at people visibly identifia...
4,2016-11,Belgium,Anti-Semitic hate crime,Threats,Antisemitisme.be,Firecrackers were thrown at people visibly ide...


In [5]:
df.columns

Index(['Date', 'Country', 'Bias motivations', 'Type of incident', 'Source',
       'Description'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27417 entries, 0 to 27416
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Date              27417 non-null  object
 1   Country           27417 non-null  object
 2   Bias motivations  27417 non-null  object
 3   Type of incident  27417 non-null  object
 4   Source            27417 non-null  object
 5   Description       24355 non-null  object
dtypes: object(6)
memory usage: 1.3+ MB


In [7]:
df.describe()

,Date,Country,Bias motivations,Type of incident,Source,Description
count,27417,27417,27417,27417,27417,24355
unique,105,53,69,6,605,21880
top,2023-12,United States of America,Racist and xenophobic hate crime,Attacks against property,Anti-Defamation League (ADL),Members of the Jewish community were threatene...
freq,1046,7058,6307,12073,6488,627


In [8]:
# Number of unique bias motivations
num_bias_motivations = df['Bias motivations'].nunique()
print(f"Number of unique bias motivations: {num_bias_motivations}")

# Number of unique countries
num_countries = df['Country'].nunique()
print(f"Number of unique countries: {num_countries}")

# Number of different sources
num_sources = df['Source'].nunique()
print(f"Number of different sources: {num_sources}")

Number of unique bias motivations: 69
Number of unique countries: 53
Number of different sources: 605


In [9]:
# Number of rows where the country is Germany
num_rows_germany = df[df['Country'] == 'Germany'].shape[0]
print(f"Number of rows for Germany: {num_rows_germany}")

Number of rows for Germany: 2028


In [10]:
# filter the dataframe to show rows where Germany get the counts of each bias motivation
bias_motivations_germany_counts = df[df['Country'] == 'Germany']['Bias motivations'].value_counts()

print("Bias motivations for Germany and their counts:")
print(bias_motivations_germany_counts)

Bias motivations for Germany and their counts:
Bias motivations
Anti-Christian hate crime                                                                                                   814
Anti-Muslim hate crime                                                                                                      386
Anti-LGBTI hate crime                                                                                                       235
Anti-Semitic hate crime                                                                                                     212
Racist and xenophobic hate crime                                                                                            200
Racist and xenophobic hate crime, Anti-Muslim hate crime                                                                     92
Gender-based hate crime, Anti-LGBTI hate crime                                                                               15
Racist and xenophobic hate crime, Anti-S

In [11]:
# Find the most common bias motivation for each country
most_common_bias_per_country = df.groupby('Country')['Bias motivations'].agg(lambda x: x.value_counts().idxmax())

print("Most common bias motivation for each country:")
print(most_common_bias_per_country)

Most common bias motivation for each country:
Country
Albania                     Gender-based hate crime, Anti-LGBTI hate crime
Armenia                                              Anti-LGBTI hate crime
Austria                                             Anti-Muslim hate crime
Azerbaijan                                           Anti-LGBTI hate crime
Belarus                                              Anti-LGBTI hate crime
Belgium                                          Anti-Christian hate crime
Bosnia and Herzegovina                    Racist and xenophobic hate crime
Bulgaria                                            Anti-Muslim hate crime
Canada                                             Anti-Semitic hate crime
Croatia                                   Racist and xenophobic hate crime
Cyprus                                    Racist and xenophobic hate crime
Czech Republic                            Racist and xenophobic hate crime
Denmark                                       

In [12]:
# Count the top 10 most common bias motivations
top_10_bias_motivations = df['Bias motivations'].value_counts().head(30)

print("Top 10 most common bias motivations:")
print(top_10_bias_motivations)

Top 10 most common bias motivations:
Bias motivations
Racist and xenophobic hate crime                                                     6307
Anti-Semitic hate crime                                                              6070
Anti-Christian hate crime                                                            5075
Anti-LGBTI hate crime                                                                4521
Anti-Muslim hate crime                                                               2188
Racist and xenophobic hate crime, Anti-Muslim hate crime                              696
Gender-based hate crime, Anti-LGBTI hate crime                                        560
Racist and xenophobic hate crime, Anti-Semitic hate crime                             356
Anti-Roma hate crime                                                                  348
Gender-based hate crime                                                               286
Racist and xenophobic hate crime, Anti-Christi

In [13]:

countries_of_interest = ['France', 'Italy', 'Spain', 'Portugal', 'Germany', 'Belgium', 'Austria', 'Switzerland', 'Netherlands', 'Denmark']

filtered_df = df[df['Country'].isin(countries_of_interest)]

# most common bias motivation for each of main euro countries
most_common_bias_selected_countries = (
    filtered_df.groupby('Country')['Bias motivations']
    .agg(lambda x: x.value_counts().idxmax())
)

print("Most reported bias motivation for each specified country:")
print(most_common_bias_selected_countries)

Most reported bias motivation for each specified country:
Country
Austria                  Anti-Muslim hate crime
Belgium               Anti-Christian hate crime
Denmark                 Anti-Semitic hate crime
France                Anti-Christian hate crime
Germany               Anti-Christian hate crime
Italy          Racist and xenophobic hate crime
Netherlands             Anti-Semitic hate crime
Portugal                  Anti-LGBTI hate crime
Spain                 Anti-Christian hate crime
Switzerland    Racist and xenophobic hate crime
Name: Bias motivations, dtype: object


In [14]:

anti_christian_germany_df = df[(df['Country'] == 'Germany') & (df['Bias motivations'] == 'Anti-Christian hate crime')]

anti_christian_descriptions = anti_christian_germany_df['Description'].head(10)
print("Anti-Christian hate crimes in Germany:")
print(anti_christian_descriptions)

Anti-Christian hate crimes in Germany:
933    A male refugee was subjected to death threats ...
940    The baby of an Eritrean Christian woman was su...
952    A male Christian refugee was stabbed in the ba...
964    Assyrian refugees were threatened in their ref...
975    A Lutheran refugee was physically assaulted an...
980    Six Christian refugees were threatened by a la...
983    A Catholic monument was vandalized with pink p...
985    A Christian academy was vandalized with paint ...
990    A religious statue was removed from a Catholic...
994    A religious statue was damaged twice within a ...
Name: Description, dtype: object


In [15]:
df.head()

,Date,Country,Bias motivations,Type of incident,Source,Description
0,2016-11,Belgium,Anti-Semitic hate crime,Violent attacks against people,Antisemitisme.be,A man visibly identifiable as Jewish was pushe...
1,2016-11,Belgium,Anti-Semitic hate crime,Threats,Antisemitisme.be,A man visibly identifiable as Jewish was chase...
2,2016-03,Belgium,Anti-Semitic hate crime,Violent attacks against people,"Anti-Defamation League (ADL), Antisemitisme.be",Stones were thrown at people visibly identifia...
3,2016-03,Belgium,Anti-Semitic hate crime,Violent attacks against people,Antisemitisme.be,Stones were thrown at people visibly identifia...
4,2016-11,Belgium,Anti-Semitic hate crime,Threats,Antisemitisme.be,Firecrackers were thrown at people visibly ide...


# Things important to consider 
1/ i need to check if the anti christian hate crimes were perpetrated against non white ppl 




In [16]:
num_types_of_incidents = df['Type of incident'].nunique()
print(f"Number of unique types of incidents: {num_types_of_incidents}")

Number of unique types of incidents: 6


In [17]:
type_of_incidents = df['Type of incident'].unique()
print("List of types of incidents:")
print(type_of_incidents)

List of types of incidents:
['Violent attacks against people' 'Threats' 'Attacks against property'
 'Threats, Violent attacks against people'
 'Attacks against property, Violent attacks against people'
 'Attacks against property, Threats']


In [18]:
# Filter the dataframe for incidents of type 'Violent attacks against people'
violent_attacks_df = df[df['Type of incident'] == 'Violent attacks against people']

# Count the occurrences for each country
violent_attacks_per_country = violent_attacks_df['Country'].value_counts()

# Find the country with the most number of violent attacks against people
most_violent_attacks_country = violent_attacks_per_country.idxmax()
most_violent_attacks_count = violent_attacks_per_country.max()

print(f"The country with the most number of violent attacks against people is {most_violent_attacks_country} with {most_violent_attacks_count} incidents.")

The country with the most number of violent attacks against people is Russian Federation with 940 incidents.


In [19]:
# Filter the dataframe for incidents of type 'Violent attacks against people'
violent_attacks_df = df[df['Type of incident'] == 'Violent attacks against people']

# Filter for the specified countries
countries_of_interest = ['France', 'Italy', 'Spain', 'Portugal', 'Germany', 'Belgium', 'Austria', 'Switzerland', 'Netherlands']
violent_attacks_df = violent_attacks_df[violent_attacks_df['Country'].isin(countries_of_interest)]

# Count the occurrences of violent attacks for each specified country
violent_attacks_per_country = violent_attacks_df['Country'].value_counts()

print("Number of violent attacks against people per country:")
print(violent_attacks_per_country)

Number of violent attacks against people per country:
Country
Italy          681
Germany        543
Spain          441
Austria        327
Switzerland    237
France         220
Netherlands     83
Belgium         70
Portugal        32
Name: count, dtype: int64


In [20]:
# Filter the dataframe for incidents of type 'Threats'
threats_df = df[df['Type of incident'] == 'Threats']

# Filter for the specified countries
countries_of_interest = ['France', 'Italy', 'Spain', 'Portugal', 'Germany', 'Belgium', 'Austria', 'Switzerland', 'Netherlands']
threats_df = threats_df[threats_df['Country'].isin(countries_of_interest)]

# Count the occurrences of threats for each specified country
threats_per_country = threats_df['Country'].value_counts()

print("Number of threats per country:")
print(threats_per_country)

Number of threats per country:
Country
Germany        344
Italy          248
Netherlands    240
Spain          199
Switzerland    141
France         127
Austria        120
Belgium         49
Portugal        26
Name: count, dtype: int64


In [21]:
countries_of_interest = ['France', 'Italy', 'Spain', 'Portugal', 'Germany', 'Belgium', 'Austria', 'Switzerland', 'Netherlands', 'Denmark']
filtered_df = df[df['Country'].isin(countries_of_interest)]
filtered_df.value_counts(['Country'])


Country    
Germany        2028
Italy          1629
France         1405
Spain          1286
Austria         926
Netherlands     567
Switzerland     525
Belgium         230
Portugal         71
Denmark          46
Name: count, dtype: int64

In [22]:
# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Extract year and month into separate columns
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

# Display the first few rows to verify the changes
df.head()

,Date,Country,Bias motivations,Type of incident,Source,Description,Year,Month
0,2016-11-01,Belgium,Anti-Semitic hate crime,Violent attacks against people,Antisemitisme.be,A man visibly identifiable as Jewish was pushe...,2016.0,11.0
1,2016-11-01,Belgium,Anti-Semitic hate crime,Threats,Antisemitisme.be,A man visibly identifiable as Jewish was chase...,2016.0,11.0
2,2016-03-01,Belgium,Anti-Semitic hate crime,Violent attacks against people,"Anti-Defamation League (ADL), Antisemitisme.be",Stones were thrown at people visibly identifia...,2016.0,3.0
3,2016-03-01,Belgium,Anti-Semitic hate crime,Violent attacks against people,Antisemitisme.be,Stones were thrown at people visibly identifia...,2016.0,3.0
4,2016-11-01,Belgium,Anti-Semitic hate crime,Threats,Antisemitisme.be,Firecrackers were thrown at people visibly ide...,2016.0,11.0


what i have seen so far :
- i need to check carrefully if the crimes against chrstians are agaisnt refugees or non white if yes i will put it as racist xenophobic
- i wanna keep the ones that are touching someone directly not "something stolen" or "tag"
- i will focus only on Bias motivations :                                                                                                
Anti-Muslim hate crime                                                                                                      
Anti-LGBTI hate crime                                                                                                       
Anti-Semitic hate crime                                                                                                     
Racist and xenophobic hate crime                                                                                            

The rest i will drop 

- still need to think about taking all of those bias and group by for germany and drop other countries 
- check into trends of threats and violent attacks 
- i need to combine incidents too DONE
- also need to combine bias 
- i need to understand a lil bit more the descriptions so i can combine bias


In [23]:
# Count the occurrences of each type of incident
incident_counts = df['Type of incident'].value_counts()

print("Number of each type of incident:")
print(incident_counts)

Number of each type of incident:
Type of incident
Attacks against property                                    12073
Violent attacks against people                               9069
Threats                                                      6269
Threats, Violent attacks against people                         4
Attacks against property, Violent attacks against people        1
Attacks against property, Threats                               1
Name: count, dtype: int64


In [24]:
# Replace specific combined incident types with the desired categories
df['Type of incident'] = df['Type of incident'].replace({
    'Attacks against property, Threats': 'Attacks against property',
    'Threats, Violent attacks against people': 'Violent attacks against people',
    'Attacks against property, Violent attacks against people': 'Violent attacks against people'
})

# Count the occurrences of each type of incident after the replacements
incident_counts = df['Type of incident'].value_counts()

print("Number of each type of incident:")
print(incident_counts)

Number of each type of incident:
Type of incident
Attacks against property          12074
Violent attacks against people     9074
Threats                            6269
Name: count, dtype: int64


In [25]:
# Filter the dataframe for incidents of type 'Violent attacks against people'
violent_attacks_df = df[df['Type of incident'] == 'Violent attacks against people']

# Count the occurrences of each bias motivation for violent attacks
violent_attacks_bias_counts = violent_attacks_df['Bias motivations'].value_counts()

print("Number of violent attacks against people related to each bias motivation:")
print(violent_attacks_bias_counts)

Number of violent attacks against people related to each bias motivation:
Bias motivations
Anti-LGBTI hate crime                                                                                      3060
Racist and xenophobic hate crime                                                                           2537
Anti-Semitic hate crime                                                                                     739
Anti-Muslim hate crime                                                                                      666
Anti-Christian hate crime                                                                                   548
Gender-based hate crime, Anti-LGBTI hate crime                                                              357
Racist and xenophobic hate crime, Anti-Muslim hate crime                                                    283
Gender-based hate crime                                                                                     203
Anti-Roma hat

In [45]:
# Define the religious hate crimes to be combined
religious_hate_crimes = [
    'Anti-Christian hate crime', 
    'Anti-Muslim hate crime', 
    'Anti-Semitic hate crime', 
    'Other hate crime based on religion or belief'
]

# Update the 'Bias motivations' column to combine religious hate crimes
df['Bias motivations'] = df['Bias motivations'].replace(religious_hate_crimes, 'Religious hate crime')

# Display the updated counts of bias motivations
updated_bias_counts = df['Bias motivations'].value_counts()
print("Updated counts of bias motivations:")
print(updated_bias_counts)

Updated counts of bias motivations:
Bias motivations
religious hate crime                                                                                            8292
Racist and xenophobic hate crime                                                                                6307
Anti-LGBTI hate crime                                                                                           4521
Racist and xenophobic hate crime, Anti-Muslim hate crime                                                         696
Gender-based hate crime, Anti-LGBTI hate crime                                                                   560
                                                                                                                ... 
Racist and xenophobic hate crime, Anti-Roma hate crime, Anti-Muslim hate crime                                     1
Other hate crime based on religion or belief, Gender-based hate crime, Anti-LGBTI hate crime                       1
Racist and 

/var/folders/78/fvr_pjz530z18q7trgx97kf00000gn/T/ipykernel_1406/2774568665.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Bias motivations'] = df['Bias motivations'].replace(religious_hate_crimes, 'Religious hate crime')


In [48]:
# Define the gender-based and anti-LGBTI hate crimes to be combined
gender_and_lgbti_hate_crimes = [
    'Gender-based hate crime', 
    'Anti-LGBTI hate crime',
    'Gender-based hate crime; Anti-LGBTI hate crime'
]

# Update the 'Bias motivations' column to combine gender-based and anti-LGBTI hate crimes
df['Bias motivations'] = df['Bias motivations'].replace(gender_and_lgbti_hate_crimes, 'Gender and Anti-LGBTI hate crime')

# Display the updated counts of bias motivations
updated_bias_counts = df['Bias motivations'].value_counts()
print("Updated counts of bias motivations:")
print(updated_bias_counts)

Updated counts of bias motivations:
Bias motivations
religious hate crime                                                                                            8292
Racist and xenophobic hate crime                                                                                6307
Gender and Anti-LGBTI hate crime                                                                                4807
Racist and xenophobic hate crime, Anti-Muslim hate crime                                                         696
Gender-based hate crime, Anti-LGBTI hate crime                                                                   560
                                                                                                                ... 
Racist and xenophobic hate crime, Anti-Roma hate crime, Anti-Muslim hate crime                                     1
Other hate crime based on religion or belief, Gender-based hate crime, Anti-LGBTI hate crime                       1
Racist and 

In [51]:
# Filter out bias motivations that count less than 10 time
filtered_bias_counts = df['Bias motivations'].value_counts()
bias_to_keep = filtered_bias_counts[filtered_bias_counts > 10].index

# Update the dataframe to keep only the rows with the specified bias motivations
df = df[df['Bias motivations'].isin(bias_to_keep)]

# Display the updated counts of bias motivations
updated_bias_counts = df['Bias motivations'].value_counts()
print("Updated counts of bias motivations:")
print(updated_bias_counts)

Updated counts of bias motivations:
Bias motivations
religious hate crime                                                                 8292
Racist and xenophobic hate crime                                                     6307
Gender and Anti-LGBTI hate crime                                                     4807
Racist and xenophobic hate crime, Anti-Muslim hate crime                              696
Gender-based hate crime, Anti-LGBTI hate crime                                        560
Racist and xenophobic hate crime, Anti-Semitic hate crime                             356
Anti-Roma hate crime                                                                  348
Racist and xenophobic hate crime, Anti-Christian hate crime                           136
Racist and xenophobic hate crime, Anti-LGBTI hate crime                               131
Racist and xenophobic hate crime, Gender-based hate crime                              87
Disability hate crime                          

In [53]:
# Filter the counts for "Racist and xenophobic hate crime" that are less than 62
racist_xenophobic_counts = df[df['Bias motivations'].str.contains('Racist and xenophobic hate crime')]['Bias motivations'].value_counts()
to_combine = racist_xenophobic_counts[racist_xenophobic_counts < 62].index

# Replace the values in the dataframe
df['Bias motivations'] = df['Bias motivations'].replace(to_combine, 'Racist and xenophobic hate crime, Other')

# Display the updated counts of bias motivations
updated_bias_counts = df['Bias motivations'].value_counts()
print("Updated counts of bias motivations:")
print(updated_bias_counts)

Updated counts of bias motivations:
Bias motivations
religious hate crime                                           8292
Racist and xenophobic hate crime                               6307
Gender and Anti-LGBTI hate crime                               4807
Racist and xenophobic hate crime, Anti-Muslim hate crime        696
Gender-based hate crime, Anti-LGBTI hate crime                  560
Racist and xenophobic hate crime, Anti-Semitic hate crime       356
Anti-Roma hate crime                                            348
Racist and xenophobic hate crime, Other                         218
Racist and xenophobic hate crime, Anti-Christian hate crime     136
Racist and xenophobic hate crime, Anti-LGBTI hate crime         131
Racist and xenophobic hate crime, Gender-based hate crime        87
Disability hate crime                                            76
Anti-Muslim hate crime, Gender-based hate crime                  62
Anti-Semitic hate crime, Anti-LGBTI hate crime                 

In [54]:
# Update the 'Bias motivations' column to categorize everything that starts with "racist" into "Racist and xenophobic hate crime"
df['Bias motivations'] = df['Bias motivations'].replace(
    to_replace=r'^Racist.*', 
    value='Racist and xenophobic hate crime', 
    regex=True
)

# Display the updated counts of bias motivations
updated_bias_counts = df['Bias motivations'].value_counts()
print("Updated counts of bias motivations:")
print(updated_bias_counts)

Updated counts of bias motivations:
Bias motivations
religious hate crime                                  8292
Racist and xenophobic hate crime                      7931
Gender and Anti-LGBTI hate crime                      4807
Gender-based hate crime, Anti-LGBTI hate crime         560
Anti-Roma hate crime                                   348
Disability hate crime                                   76
Anti-Muslim hate crime, Gender-based hate crime         62
Anti-Semitic hate crime, Anti-LGBTI hate crime          52
Anti-Semitic hate crime, Gender-based hate crime        36
Anti-Semitic hate crime, Anti-Christian hate crime      24
Disability hate crime, Anti-LGBTI hate crime            22
Anti-Semitic hate crime, Anti-Muslim hate crime         12
Name: count, dtype: int64


In [55]:
# Update the 'Bias motivations' column to categorize everything that starts with "Anti-Semitic" and "Anti-Muslim" into "religious hate crime"
df['Bias motivations'] = df['Bias motivations'].replace(
    to_replace=r'^Anti-(Semitic|Muslim).*', 
    value='religious hate crime', 
    regex=True
)

# Display the updated counts of bias motivations
updated_bias_counts = df['Bias motivations'].value_counts()
print("Updated counts of bias motivations:")
print(updated_bias_counts)

Updated counts of bias motivations:
Bias motivations
religious hate crime                              8478
Racist and xenophobic hate crime                  7931
Gender and Anti-LGBTI hate crime                  4807
Gender-based hate crime, Anti-LGBTI hate crime     560
Anti-Roma hate crime                               348
Disability hate crime                               76
Disability hate crime, Anti-LGBTI hate crime        22
Name: count, dtype: int64


In [56]:
# Update the 'Bias motivations' column to categorize "Anti-Roma hate crime" as "Racist and xenophobic hate crime"
df['Bias motivations'] = df['Bias motivations'].replace('Anti-Roma hate crime', 'Racist and xenophobic hate crime')

# Display the updated counts of bias motivations
updated_bias_counts = df['Bias motivations'].value_counts()
print("Updated counts of bias motivations:")
print(updated_bias_counts)

Updated counts of bias motivations:
Bias motivations
religious hate crime                              8478
Racist and xenophobic hate crime                  8279
Gender and Anti-LGBTI hate crime                  4807
Gender-based hate crime, Anti-LGBTI hate crime     560
Disability hate crime                               76
Disability hate crime, Anti-LGBTI hate crime        22
Name: count, dtype: int64


In [58]:
# Define the gender-based and anti-LGBTI hate crimes to be combined
gender_and_lgbti_hate_crimes = [
    'Gender-based hate crime, Anti-LGBTI hate crime'
]

# Update the 'Bias motivations' column to combine gender-based and anti-LGBTI hate crimes
df['Bias motivations'] = df['Bias motivations'].replace(gender_and_lgbti_hate_crimes, 'Gender and Anti-LGBTI hate crime')

# Display the updated counts of bias motivations
updated_bias_counts = df['Bias motivations'].value_counts()
print("Updated counts of bias motivations:")
print(updated_bias_counts)

Updated counts of bias motivations:
Bias motivations
religious hate crime                            8478
Racist and xenophobic hate crime                8279
Gender and Anti-LGBTI hate crime                5367
Disability hate crime                             76
Disability hate crime, Anti-LGBTI hate crime      22
Name: count, dtype: int64


In [60]:
# Filter out bias motivations that count less than 76 times
filtered_bias_counts = df['Bias motivations'].value_counts()
bias_to_keep = filtered_bias_counts[filtered_bias_counts > 76].index

# Update the dataframe to keep only the rows with the specified bias motivations
df = df[df['Bias motivations'].isin(bias_to_keep)]

# Display the updated counts of bias motivations
updated_bias_counts = df['Bias motivations'].value_counts()
print("Updated counts of bias motivations:")
print(updated_bias_counts)

Updated counts of bias motivations:
Bias motivations
religious hate crime                8478
Racist and xenophobic hate crime    8279
Gender and Anti-LGBTI hate crime    5367
Name: count, dtype: int64


In [61]:
num_rows = df.shape[0]
print(f"Number of rows in the dataframe: {num_rows}")

Number of rows in the dataframe: 22124


In [64]:
df.head(30)

,Date,Country,Bias motivations,Type of incident,Source,Description,Year,Month,Bias Combined
0,2016-11-01,Belgium,religious hate crime,Violent attacks against people,Antisemitisme.be,A man visibly identifiable as Jewish was pushe...,2016.0,11.0,Religious-related hate
1,2016-11-01,Belgium,religious hate crime,Threats,Antisemitisme.be,A man visibly identifiable as Jewish was chase...,2016.0,11.0,Religious-related hate
2,2016-03-01,Belgium,religious hate crime,Violent attacks against people,"Anti-Defamation League (ADL), Antisemitisme.be",Stones were thrown at people visibly identifia...,2016.0,3.0,Religious-related hate
3,2016-03-01,Belgium,religious hate crime,Violent attacks against people,Antisemitisme.be,Stones were thrown at people visibly identifia...,2016.0,3.0,Religious-related hate
4,2016-11-01,Belgium,religious hate crime,Threats,Antisemitisme.be,Firecrackers were thrown at people visibly ide...,2016.0,11.0,Religious-related hate
5,2016-11-01,Belgium,religious hate crime,Threats,Antisemitisme.be,Firecrackers were thrown at people visibly ide...,2016.0,11.0,Religious-related hate
6,2016-01-01,Belgium,religious hate crime,Violent attacks against people,Antisemitisme.be,"A lesbian Jewish woman was assaulted, threaten...",2016.0,1.0,"Anti-Semitic hate crime, Anti-LGBTI hate crime"
7,2016-12-01,Belgium,religious hate crime,Attacks against property,Antisemitisme.be,A swastika was spray-painted on the sidewalk i...,2016.0,12.0,Religious-related hate
8,2016-01-01,Belgium,religious hate crime,Attacks against property,Antisemitisme.be,Anti-Semitic graffiti were spray-painted on a ...,2016.0,1.0,Religious-related hate
9,2016-03-01,Belgium,religious hate crime,Attacks against property,"Anti-Defamation League (ADL), Antisemitisme.be",Anti-Semitic graffiti were spray-painted on a ...,2016.0,3.0,Religious-related hate


In [67]:
if 'Bias Combined' in df.columns:
    df = df.drop(columns=['Bias Combined'])

# Display the first few rows to verify the changes
df.head(20)

,Date,Country,Bias motivations,Type of incident,Source,Description,Year,Month
0,2016-11-01,Belgium,religious hate crime,Violent attacks against people,Antisemitisme.be,A man visibly identifiable as Jewish was pushe...,2016.0,11.0
1,2016-11-01,Belgium,religious hate crime,Threats,Antisemitisme.be,A man visibly identifiable as Jewish was chase...,2016.0,11.0
2,2016-03-01,Belgium,religious hate crime,Violent attacks against people,"Anti-Defamation League (ADL), Antisemitisme.be",Stones were thrown at people visibly identifia...,2016.0,3.0
3,2016-03-01,Belgium,religious hate crime,Violent attacks against people,Antisemitisme.be,Stones were thrown at people visibly identifia...,2016.0,3.0
4,2016-11-01,Belgium,religious hate crime,Threats,Antisemitisme.be,Firecrackers were thrown at people visibly ide...,2016.0,11.0
5,2016-11-01,Belgium,religious hate crime,Threats,Antisemitisme.be,Firecrackers were thrown at people visibly ide...,2016.0,11.0
6,2016-01-01,Belgium,religious hate crime,Violent attacks against people,Antisemitisme.be,"A lesbian Jewish woman was assaulted, threaten...",2016.0,1.0
7,2016-12-01,Belgium,religious hate crime,Attacks against property,Antisemitisme.be,A swastika was spray-painted on the sidewalk i...,2016.0,12.0
8,2016-01-01,Belgium,religious hate crime,Attacks against property,Antisemitisme.be,Anti-Semitic graffiti were spray-painted on a ...,2016.0,1.0
9,2016-03-01,Belgium,religious hate crime,Attacks against property,"Anti-Defamation League (ADL), Antisemitisme.be",Anti-Semitic graffiti were spray-painted on a ...,2016.0,3.0


In [71]:
num_incidents_germany = df[df['Country'] == 'Germany'].shape[0]
print(f"Number of incidents in Germany: {num_incidents_germany}")

Number of incidents in Germany: 1207


In [72]:
# Filter the dataframe to keep only the rows where the country is Germany
df = df[df['Country'] == 'Germany']

# Display the first few rows to verify the changes
df.head()

,Date,Country,Bias motivations,Type of incident,Source,Description,Year,Month
756,2016-03-01,Germany,religious hate crime,Violent attacks against people,Department for Research and Information on Ant...,A group of Spanish tourists was spat at and su...,2016.0,3.0
757,2016-04-01,Germany,religious hate crime,Attacks against property,Department for Research and Information on Ant...,Plates commemorating female Jewish students we...,2016.0,4.0
758,2016-04-01,Germany,religious hate crime,Attacks against property,Department for Research and Information on Ant...,A sticker denying the Holocaust was left on a ...,2016.0,4.0
759,2016-05-01,Germany,religious hate crime,Attacks against property,Department for Research and Information on Ant...,"Four stolpersteine, stones commemorating Holoc...",2016.0,5.0
760,2016-06-01,Germany,religious hate crime,Attacks against property,Department for Research and Information on Ant...,"Several stolpersteine, stones commemorating Ho...",2016.0,6.0


In [73]:
# Count the number of incidents related to "Gender and Anti-LGBTI hate crime"
gender_and_lgbti_count = df[df['Bias motivations'] == 'Gender and Anti-LGBTI hate crime'].shape[0]

print(f"Number of Gender and Anti-LGBTI hate crime incidents: {gender_and_lgbti_count}")

Number of Gender and Anti-LGBTI hate crime incidents: 254
